In [1]:
using Revise
using FFTW, Optim, Images, ImageView, TestImages
using Noise
using DeconvOptim
using Statistics
using BenchmarkTools
using Deconvolution
using Zygote
using Tullio
using Plots
#BenchmarkTools.DEFAULT_PARAMETERS.samples = 10
#BenchmarkTools.DEFAULT_PARAMETERS.seconds = 100

┌ Info: Precompiling Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1260
┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1260
┌ Info: Precompiling ImageView [86fae568-95e7-573e-a6b2-d8a6b900c9ef]
└ @ Base loading.jl:1260
┌ Info: Precompiling Noise [81d43f40-5267-43b7-ae1c-8b967f377efa]
└ @ Base loading.jl:1260
┌ Info: Precompiling DeconvOptim [03e7cd2f-1a03-4ea9-b59b-760a446df67f]
└ @ Base loading.jl:1260
┌ Info: Precompiling Deconvolution [41ba435c-a500-5816-a783-a9707c6f5f3a]
└ @ Base loading.jl:1260
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


In [2]:
function imshowv(args...)
    imshow(cat(args..., dims=3))
end


function lucy2(observed::AbstractArray, psf::AbstractArray; iterations::Integer = 1)
    @assert size(observed) == size(psf)
    @assert iterations >= 0

    psf_ft = fft(psf)
    psf_ft_conj = conj.(psf_ft)

    function lucystep(e)
        return e .* real(ifft(fft(observed ./ ifft(fft(e) .* psf_ft)) .* psf_ft_conj)) 
    end

    estimated = real(ifft(fft(observed) .* psf_ft))
    for i in 1:iterations
        estimated = lucystep(estimated)
    end

    return estimated
end

lucy2 (generic function with 1 method)

In [3]:
img2 = channelview(load("../DeconvOptim/resolution_512.tif"))
img = convert(Array{Float32}, channelview(load("../DeconvOptim/resolution_512.tif")))
#img = convert(Array{Float32}, testimage("fabio_gray_512"))
#img = convert(Array{Float32}, channelview(testimage("peppers_gray"))[1, :, :])
r = 30
psf = convert(Array{Float32}, DeconvOptim.generate_psf(img, r))
otf = rfft(psf)

img_b = max.(0, DeconvOptim.conv_real_otf(img, otf))
img_noisy = poisson(img_b, 100)

imshowv(img, img_b, img_noisy);

┌ Info: Precompiling ImageMagick [6218d12a-5da1-5696-b52f-db25d2ecc6d1]
└ @ Base loading.jl:1260


In [4]:
@time res_lr1 = lucy(img_noisy, psf, iterations=50);
@time res_lr2 = lucy2(img_noisy, psf, iterations=50);
imshowv(res_lr1, res_lr2)

  2.103770 seconds (1.72 M allocations: 987.636 MiB, 2.99% gc time)
  1.446294 seconds (202.08 k allocations: 923.327 MiB, 1.98% gc time)


Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>110: "map(clim-mapped image, inpu…
  "annotations" => 59: "input-22" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 58: "CLim" = CLim{Float32}(0.0, 1.72847) CLim{Float32} 

In [9]:
reg = Tikhonov(sum_dims=[1, 2], weights=[1, 1], λ=0.1, mode="laplace")
@time resTikho, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=20))
print(o)


reg = DeconvOptim.GR(sum_dims=[1, 2], weights=[1, 1], λ=0.05, mode="central")
@time resGR, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=20))
print(o)


reg = DeconvOptim.TV(sum_dims=[1, 2], weights=[1, 1], λ=0.01, step=1, mode="central")
@time resTV, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=20))
print(o)

@time res, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=nothing,
        options=Optim.Options(iterations=20))
print(o)

  1.559424 seconds (1.14 M allocations: 1.347 GiB, 3.34% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [1.14e-03, 7.93e-04, 3.40e-04,  ...]
    Minimum:   2.052756e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.94e-02, 4.99e-02, 5.05e-02,  ...]

 * Convergence measures
    |x - x'|               = 1.54e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.37e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.85e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 9.00e-06 ≰ 0.0e+00
    |g(x)|                 = 1.03e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    20
    f(x) calls:    64
    ∇f(x) calls:   64
  1.645343 seconds (1.47 M allocations: 1.303 GiB, 2.79% gc time)
 * Status: success

 * Candidate solution
    Minimizer: [-7.53e-05, -2.54e-05, 4.22e-05,  ...]
    Minimum:   2.055421e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.94e-02, 4.99e-02, 5.05e-02,  ...]

 *

In [11]:
imshowv(img, res[:, :], resTikho[:, :], resGR[:, :], resTV[:, :], res_lr2, img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>220: "map(clim-mapped image, inpu…
  "annotations" => 169: "input-79" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 168: "CLim" = CLim{Float32}(0.0, 1.0) CLim{Float32} 

In [151]:
reg = DeconvOptim.GR(sum_dims=[1, 2], weights=[1, 1], λ=0.05, mode="central")

@time resGR, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=10))
print(o)

  1.192040 seconds (1.46 M allocations: 892.854 MiB, 4.17% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [7.24e-05, 1.55e-04, 2.95e-04,  ...]
    Minimum:   2.055268e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.88e-02, 4.88e-02, 4.87e-02,  ...]

 * Convergence measures
    |x - x'|               = 1.17e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.06e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.07e-07 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.47e-06 ≰ 0.0e+00
    |g(x)|                 = 3.00e-08 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    10
    f(x) calls:    37
    ∇f(x) calls:   37


In [152]:
imshowv(img, res[:, :], resTikho_sgs[:, :], resGR[:, :], 
        res_lr2, img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>865: "map(clim-mapped image, inpu…
  "annotations" => 814: "input-328" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 813: "CLim" = CLim{Float32}(0.0, 1.0) CLim{Float32} 

In [180]:
reg = DeconvOptim.TV(sum_dims=[1, 2], weights=[1, 1], λ=0.01, step=2, mode="central")


@time resTV, o, r0 = deconvolution(img_noisy, psf, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=reg,
        options=Optim.Options(iterations=10))
print(o)


  1.007530 seconds (1.05 M allocations: 835.226 MiB, 2.46% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-4.56e-04, -2.36e-04, 4.94e-05,  ...]
    Minimum:   2.057313e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.88e-02, 4.88e-02, 4.87e-02,  ...]

 * Convergence measures
    |x - x'|               = 4.58e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.38e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.26e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.07e-05 ≰ 0.0e+00
    |g(x)|                 = 3.35e-07 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    10
    f(x) calls:    37
    ∇f(x) calls:   37


In [181]:
imshowv(img, resTV[:, :], res_lr2,  img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>1360: "map(clim-mapped image, inp…
  "annotations" => 1309: "input-499" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 1308: "CLim" = CLim{Float32}(0.0, 1.0) CLim{Float32} 

In [174]:
img3d = cat(img_noisy, img_noisy, img_noisy, dims=3);
img4d = cat(img3d, img3d ./ 2, img3d .* 2, dims=4);
img5d = reshape(img4d, (size(img4d)..., 1))

psf3d = cat(psf, zeros(size(img)), zeros(size(img)), dims=3)
psf4d = cat(psf, psf, psf, dims=3)
psf5d = reshape(psf4d, (size(psf4d)..., 1))
#imshow(img3d)
print(size(psf5d))
print(size(img5d))


@time resGR, o, r0 = deconvolution(img5d, psf3d, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=DeconvOptim.GR(λ=0.02, mode="central"),
        options=Optim.Options(iterations=10))
print(o)

@time resGR_old, o, r0 = deconvolution(img5d, psf3d, 
        lossf=Poisson(), mappingf=Non_negative(), regularizerf=DeconvOptim.GR(λ=0.02, mode="forward"),
        options=Optim.Options(iterations=10))
print(o)


(512, 512, 3, 1)(512, 512, 3, 3, 1)  8.109354 seconds (4.34 M allocations: 8.408 GiB, 5.47% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-2.83e-03, -2.64e-03, -2.26e-03,  ...]
    Minimum:   2.111696e-01

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [4.88e-02, 4.88e-02, 4.87e-02,  ...]

 * Convergence measures
    |x - x'|               = 6.80e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.20e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.77e-07 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.26e-06 ≰ 0.0e+00
    |g(x)|                 = 7.26e-06 ≰ 1.0e-08

 * Work counters
    Seconds run:   6  (vs limit Inf)
    Iterations:    10
    f(x) calls:    36
    ∇f(x) calls:   36
  7.280548 seconds (1.45 M allocations: 8.271 GiB, 5.48% gc time)
 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [-4.78e-03, -4.59e-03, -4.23e-03,  ...]
    Minimum:   2.110692e-01

 * Found with
    Alg

In [175]:
imshowv(img, resGR_old[:,:,1,1,1], resGR[:,:,2,1,1], res_lr2,  img_noisy)

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>1195: "map(clim-mapped image, inp…
  "annotations" => 1144: "input-442" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 1143: "CLim" = CLim{Float32}(0.0, 1.0) CLim{Float32} 